In [71]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/BigData

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/BigData


In [72]:
#import
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE, Isomap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, Normalizer
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
import random
import copy

In [73]:
df = pd.read_csv('./results.csv')
#date 형식 변환
df['date'] = [i.replace('-','') for i in df['date']]

df_old =  pd.read_csv('./results.csv')

In [74]:
#all_list : tournament, country, city, home_team, away_team
all_list = ((df['tournament'].unique()).tolist() + (df['country'].unique()).tolist() + (df['city'].unique()).tolist() + (df['home_team'].unique()).tolist() + (df['away_team'].unique()).tolist())

uni_len = '***'.join(all_list)
nat_enc = tf.keras.preprocessing.text.Tokenizer(num_words=len(all_list),
    filters='',
    lower=False,
    split = '***')
nat_enc.fit_on_texts([uni_len])
nat_dic = nat_enc.word_index

In [75]:
#값을 enumerate로 변환
for step, val in enumerate(df['tournament']):
    df['tournament'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['city']):
    df['city'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['country']):
    df['country'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['home_team']):
    df['home_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['away_team']):
    df['away_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['neutral']):
    if df['neutral'].iloc[step] == False:
        df['neutral'].iloc[step] = 0
    if df['neutral'].iloc[step] == True:
        df['neutral'].iloc[step] = 1

In [76]:
#df_new
df_new = df
#len의 역할?
temp_tor = []
for i in df_old['tournament']:
    temp_tor.append(len(i))
temp_away = []
for i in df_old['away_team']:
    temp_away.append(len(i))
temp_home = []
for i in df_old['home_team']:
    temp_home.append(len(i))
df_new['tor_len'] = temp_tor
df_new['home_len'] = temp_home
df_new['away_len'] = temp_away

#df_new : date 형태 변환 ex. 18721130 -> -4.870880
exp = df_new['date'].to_numpy()
scaler = StandardScaler()
exp_2 = scaler.fit_transform(exp.reshape(-1,1))
df_new['date'] = (np.ravel(exp_2)).tolist()
# 결측값 있는 행 제거
df_new = df_new.dropna()

df_label1 = df_new['home_score']
df_label2 = df_new['away_score']
df_new = df_new.drop(['home_score', 'away_score'], axis = 1)


In [77]:
x_train = df_new.to_numpy()

In [78]:
#df_label
df_train = pd.DataFrame(data=x_train, columns=df_new.columns)
df_label = pd.concat([df_label1, df_label2],axis = 1)
df_label.columns = ['home_score', 'away_score']
#win column 추가
win = []
for h, a in zip(df_label['home_score'], df_label['away_score']):
    if h > a:
        win.append('home_win')
    if a > h:
        win.append('away_win')
    if a == h:
        win.append('draw')
df_label['win'] = win


In [79]:
df_label

,home_score,away_score,win
0,0.0,0.0,draw
1,4.0,2.0,home_win
2,2.0,1.0,home_win
3,2.0,2.0,draw
4,3.0,0.0,home_win
...,...,...,...
44099,2.0,1.0,home_win
44100,0.0,5.0,away_win
44101,2.0,0.0,home_win
44102,0.0,0.0,draw


In [80]:
#x_train = df_new.to_numpy()
Norm = Normalizer()
x_temp = Norm.fit_transform(x_train)
df_temp = pd.DataFrame(data=x_temp, columns=df_new.columns)

In [81]:
df_corr = df_temp.corr()

In [82]:
df_label.drop('win', axis=1)

,home_score,away_score
0,0.0,0.0
1,4.0,2.0
2,2.0,1.0
3,2.0,2.0
4,3.0,0.0
...,...,...
44099,2.0,1.0
44100,0.0,5.0
44101,2.0,0.0
44102,0.0,0.0


In [83]:
df_temp.drop('city', axis=1)

,date,home_team,away_team,tournament,country,neutral,tor_len,home_len,away_len
0,-0.008538,0.061351,0.063104,0.534631,0.061351,0.0,0.014023,0.014023,0.012270
1,-0.008459,0.063005,0.061254,0.533789,0.063005,0.0,0.014001,0.012251,0.014001
2,-0.008401,0.061351,0.063104,0.534631,0.061351,0.0,0.014023,0.014023,0.012270
3,-0.008317,0.063005,0.061255,0.533789,0.063005,0.0,0.014001,0.012251,0.014001
4,-0.008259,0.061351,0.063104,0.534632,0.061351,0.0,0.014023,0.014023,0.012270
...,...,...,...,...,...,...,...,...,...
44099,0.000498,0.029217,0.014203,0.123765,0.029217,0.0,0.003246,0.002435,0.003246
44100,0.001077,0.152813,0.036886,0.267862,0.152813,0.0,0.007026,0.017565,0.007904
44101,0.000851,0.135273,0.133885,0.211581,0.135273,0.0,0.005550,0.006937,0.006937
44102,0.001753,0.101466,0.141481,0.451597,0.101466,0.0,0.014291,0.012862,0.010004


In [84]:
df_label

,home_score,away_score,win
0,0.0,0.0,draw
1,4.0,2.0,home_win
2,2.0,1.0,home_win
3,2.0,2.0,draw
4,3.0,0.0,home_win
...,...,...,...
44099,2.0,1.0,home_win
44100,0.0,5.0,away_win
44101,2.0,0.0,home_win
44102,0.0,0.0,draw


In [85]:
y_es = df_label.drop('win', axis=1).to_numpy()
x_es = df_temp.drop('city', axis=1).to_numpy()
x_t, x_val , y_t, y_val = train_test_split(x_es, y_es, random_state=28)

In [86]:
#x_t=pd.DataFrame(data=x_temp)
#x_t

In [87]:
params = {
    "estimator__colsample_bylevel": uniform(0.7, 0.3),
    "estimator__gamma": uniform(0, 0.5),
    "estimator__learning_rate": uniform(0.003, 0.3),
    "estimator__max_depth": randint(2, 6),
    "estimator__n_estimators": randint(100, 500), 
    "estimator__subsample": uniform(0.6, 0.4)
}

In [88]:
#Multioutput regression are regression problems that involve predicting two or more numerical values given an input example.
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', eval_metric=['mae'], random_state = 1, use_label_encoder=False))

In [89]:
#search : 하이퍼 파라미터 튜닝
search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state =1, n_iter=5, cv=3,
                            verbose=0, n_jobs=1, return_train_score=True) 
search.fit(x_t, y_t) 

best_param = search.best_params_
print('-')
print(search.best_score_)
print(best_param)
print('-')

-
0.10852745493991223
{'estimator__colsample_bylevel': 0.7277015784306393, 'estimator__gamma': 0.09313010568883545, 'estimator__learning_rate': 0.10666821811291433, 'estimator__max_depth': 4, 'estimator__n_estimators': 381, 'estimator__subsample': 0.8155266936013428}
-


In [90]:
#RandomizedSearchCV로 찾은 best param을 모델에 적용
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror',eval_metric=['mae'], random_state = 1, 
                              use_label_encoder=False, **best_param)).fit(x_t, y_t) 

In [91]:
#MAE : MSE와 유사한 loss function
#이 코드에서 loss function이 적용되는 부분이 어디지? 못 찾았음
print('MAE: ',mean_absolute_error(y_val, xgb_model.predict(x_val)))

MAE:  1.0839428344574897


In [92]:
#K-fold 교차 검증:
#전체 훈련 데이터를 K개의 그룹으로 묶고, 그중 한 그룹을 검증셋으로 사용합니다. 
#이때 각각의 모든 데이터 그룹이 적어도 한 번씩은 검증셋으로 사용되어야 합니다. 
#K-fold 교차 검증을 사용함으로써 검정 데이터에 모델이 과적합 되는 것을 막을 수 있습니다. 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(xgb_model, x_val, y_val, scoring='neg_mean_absolute_error', cv=folds, verbose=0)
print('KFold_Neg_MAE: ',np.mean(scores))

KFold_Neg_MAE:  -1.0911797128265908


TEST 32

In [93]:

#예선 팀 조 편성 결과 입력
Group_A= ["Uruguay","Russia","Saudi Arabia","Egypt"]
Group_B= ["Portugal","Spain","Morocco","Iran"]
Group_C= ["France","Denmark","Australia","Peru"]
Group_D= ["Argentina","Croatia","Iceland","Nigeria"]
Group_E= ["Brazil","Switzerland","Costa Rica","Serbia"]
Group_F= ["South Korea","Mexico","Sweden","Germany"]
Group_G= ["Belgium","England","Panama","Tunisia"]
Group_H= ["Senegal","Japan","Poland","Colombia"]
Groups={"Group A":Group_A,"Group B":Group_B,"Group C":Group_C,"Group D":Group_D,"Group E":Group_E,"Group F":Group_F,"Group G":Group_G,"Group H":Group_H}


In [94]:
def create_input(home_team, away_team):
  #인풋으로 home_team, away_team만 주면 되는가? 아니면 별도의 벡터 생성 필요?
  ind_list = [home_team, away_team]
  return ind_list#input_vector

In [95]:
def create_params():
  seed = x_train[random.randint(0,x_train.shape[0])].reshape(1,-1)
  seed = np.delete(seed, 4, axis=1)
  params = {
    'autofill': True,
    'Norm': Norm,
    'seed': seed,
    'model': xgb_model
  }
  return params

In [96]:
#inp_list: 사용자가 입력한 home_team, away team
def predict_score(inp_list, autofill, Norm, seed, model):
    ind_list = []
    for i in inp_list:
        ind_list.append(nat_dic.get(i))
    #seed의 목적이 뭐지?
    seed[:,1] = ind_list[0]
    seed[:,2] = ind_list[1]
    x_test = Norm.fit_transform(seed)
    
    pred = model.predict(x_test)
    return np.ravel(pred).tolist()

In [97]:
def select_winning_team(ans, qual): #probability_array
    pred_ans = copy.deepcopy(ans)
    #예선 : 소수점 x, 정수로 승패 판결
    if qual == True :
      #정수
      pred_ans[0] = round(pred_ans[0])
      pred_ans[1] = round(pred_ans[1])
      
    #본선
    else :
      #소수점 1자리
      pred_ans[0] = round(pred_ans[0], 1)
      pred_ans[1] = round(pred_ans[1], 1)
    
    if (pred_ans[0]>pred_ans[1]):
          out=0 # home이 우승
    elif (pred_ans[0]<pred_ans[1]):
          out=1 # away가 우승
    elif (pred_ans[0]==pred_ans[1]):
          out=2 #비김

    return out, pred_ans

In [98]:
##Group stage Matches
print("========Qualifying Games=========")
Group_standings={}
for grp_name in list(Groups.keys()):
    print(f"[{grp_name} Matches]")
    #해당 그룹의 팀들
    probable_countries=Groups[grp_name]

    team_wins_dct={}
    goal_scored_dct={}
    goal_against_dct={}
    win_dct={}
    draw_dct={}
    lost_dct={}
    
    for i in range(len(probable_countries)):
        #team i: team1, team j : team2
        j=i+1
        team_1=probable_countries[i]

        team_wins=0
        """
        win_dct['name']='win_dict'
        draw_dct['name']='draw_dict'
        lost_dct['name']='lost_dict'
        team_wins_dct['name']='team_wins_dict'
        goal_scored_dct['name']='goal_scored_dict'
        goal_against_dct['name']='goal_against_dict'
        """
        while j<len((probable_countries)):
            team_2=probable_countries[j]
            team_lst=[team_1,team_2]

            #인풋으로 home_team, away_team만 주면 되는가? 아니면 별도의 벡터 생성 필요?
            input=create_input(team_1, team_2)#np.array([[year,stadium_num,team_1_num,team_2_num]])
            params = create_params()
            #ans : 소수점 ex 1.3 : 1.5
            ans = predict_score(input, **params)
            #qual : 예선이면 true, 본선이면 false
            qual = True
           
            win, prob_lst=select_winning_team(ans, qual)
            print("applied_score : ", prob_lst,", predicted_score: ", ans)
            
            #본인 팀 스코어
            goal_scored_dct[team_1] = goal_scored_dct.get(team_1,0)+prob_lst[0]
            goal_scored_dct[team_2] = goal_scored_dct.get(team_2,0)+prob_lst[1]
            #상대 팀 스코어
            goal_against_dct[team_1] = goal_against_dct.get(team_1,0)+prob_lst[1]
            goal_against_dct[team_2] = goal_against_dct.get(team_2,0)+prob_lst[0]

            try:
                print(f" {team_1} vs {team_2} \n  {team_lst[win]} wins 🎊🏆⚽🎖️\n")
                #team1 우승
                if (win)==0:
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+2
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)
                    #team1 1승 추가
                    win_dct[team_1] = win_dct.get(team_1,0)+1
                    win_dct[team_2] = win_dct.get(team_2,0)
                    #team2 1패 추가
                    lost_dct[team_2] = lost_dct.get(team_2,0)+1
                    lost_dct[team_1] = lost_dct.get(team_1,0)
                    #비긴 팀 없음
                    draw_dct[team_2] = draw_dct.get(team_2,0)
                    draw_dct[team_1] = draw_dct.get(team_1,0)

                elif (win)==1:
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+2
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)
                    #team2 1승 추가
                    win_dct[team_2] = win_dct.get(team_2,0)+1
                    win_dct[team_1] = win_dct.get(team_1,0)
                    #team1 1패 추가
                    lost_dct[team_1] = lost_dct.get(team_1,0)+1
                    lost_dct[team_2] = lost_dct.get(team_2,0)
                    #비긴 팀 없음
                    draw_dct[team_1] = draw_dct.get(team_1,0)
                    draw_dct[team_2] = draw_dct.get(team_2,0)

            except IndexError:
              #비긴 경우
              print(f"{team_1} vs {team_2} \n  Match Draw ⚽⚽⚽\n") 
              team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+1
              team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+1
              
              #비김
              draw_dct[team_1] = draw_dct.get(team_1,0)+1
              draw_dct[team_2] = draw_dct.get(team_2,0)+1
              #승패 없음
              win_dct[team_1] = win_dct.get(team_1,0)
              lost_dct[team_1] = lost_dct.get(team_1,0)
              #승패 없음
              win_dct[team_2] = win_dct.get(team_2,0)
              lost_dct[team_2] = lost_dct.get(team_2,0)


            j=j+1
    
    group_results=[win_dct,draw_dct,lost_dct,team_wins_dct,goal_scored_dct,goal_against_dct]
    Group_standings[grp_name]=group_results


========Qualifying Games=========
[Group A Matches]
applied_score :  [2, 2] , predicted_score:  [2.0262205600738525, 1.8895676136016846]
Uruguay vs Russia 
  Match Draw ⚽⚽⚽

applied_score :  [3, 1] , predicted_score:  [3.4232659339904785, 0.901126503944397]
 Uruguay vs Saudi Arabia 
  Uruguay wins 🎊🏆⚽🎖️

applied_score :  [2, 1] , predicted_score:  [2.264749765396118, 1.28861665725708]
 Uruguay vs Egypt 
  Uruguay wins 🎊🏆⚽🎖️

applied_score :  [1, 1] , predicted_score:  [1.3754318952560425, 1.1002250909805298]
Russia vs Saudi Arabia 
  Match Draw ⚽⚽⚽

applied_score :  [3, 2] , predicted_score:  [3.3687093257904053, 1.866811990737915]
 Russia vs Egypt 
  Russia wins 🎊🏆⚽🎖️

applied_score :  [3, 1] , predicted_score:  [2.9972708225250244, 1.3932757377624512]
 Saudi Arabia vs Egypt 
  Saudi Arabia wins 🎊🏆⚽🎖️

[Group B Matches]
applied_score :  [2, 2] , predicted_score:  [1.8266735076904297, 1.6193909645080566]
Portugal vs Spain 
  Match Draw ⚽⚽⚽

applied_score :  [1, 3] , predicted_score:  [

In [99]:
#Display group stage results

for grp_name in list(Group_standings.keys()):

    team_wins_dct= dict(sorted(Group_standings[grp_name][3].items()))
    goal_scored_dct=dict(sorted(Group_standings[grp_name][4].items()))
    goal_against_dct=dict(sorted(Group_standings[grp_name][5].items()))
    
    win_dct=dict(sorted(Group_standings[grp_name][0].items()))
    draw_dct=dict(sorted(Group_standings[grp_name][1].items()))
    lost_dct=dict(sorted(Group_standings[grp_name][2].items()))
    
    lst_teams=list(team_wins_dct.keys())
    
    win_lst=list(win_dct.values())
    draw_lst=list(draw_dct.values())
    lost_lst=list(lost_dct.values())
    
    lst_win_count=list(team_wins_dct.values())
    goal_scored=list(goal_scored_dct.values())
    goal_against=list(goal_against_dct.values())
    goal_differance=[goal_scored[i]-goal_against[i] for i in range (len(goal_scored))]
    ranking_table=pd.DataFrame(list(zip(lst_teams,win_lst,draw_lst,lost_lst,goal_scored,goal_against,goal_differance,lst_win_count)),columns=["Team","Wins","Draw","Lost","Goal Scored","Goal Against","Goal Differance","Points"])
    ranking_table=ranking_table.sort_values("Points",ascending=False).reset_index(drop=True)
    ranking_table.index = ranking_table.index + 1
    print(f"\n\n{grp_name} Final Rankings")
    print(ranking_table.to_markdown())



Group A Final Rankings
|    | Team         |   Wins |   Draw |   Lost |   Goal Scored |   Goal Against |   Goal Differance |   Points |
|---:|:-------------|-------:|-------:|-------:|--------------:|---------------:|------------------:|---------:|
|  1 | Uruguay      |      2 |      1 |      0 |             7 |              4 |                 3 |        5 |
|  2 | Russia       |      1 |      2 |      0 |             6 |              5 |                 1 |        4 |
|  3 | Saudi Arabia |      1 |      1 |      1 |             5 |              5 |                 0 |        3 |
|  4 | Egypt        |      0 |      0 |      3 |             4 |              8 |                -4 |        0 |


Group B Final Rankings
|    | Team     |   Wins |   Draw |   Lost |   Goal Scored |   Goal Against |   Goal Differance |   Points |
|---:|:---------|-------:|-------:|-------:|--------------:|---------------:|------------------:|---------:|
|  1 | Morocco  |      3 |      0 |      0 |          

In [100]:
##Round of 16 Section_1

qualified_teams_1=[]
standings=list(Group_standings.keys())
i=0
print(f"Round of 16\n")
while i < (len(standings)):
    A_team= sorted(Group_standings[standings[i]][3].items(), key=lambda x: x[1], reverse=True)
    team_1=A_team[0][0]
    B_team= sorted(Group_standings[standings[i+1]][3].items(), key=lambda x: x[1], reverse=True)
    team_2=B_team[1][0]
    
    team_1_num=label_encoder.transform([team_1])[0]
    team_2_num=label_encoder.transform([team_2])[0]
    team_lst=[team_1,team_2]
    
    Input_vector=np.array([[year,host_num,team_1_num,team_2_num]])
    res=model.predict(Input_vector)
    win,_=select_winning_team(res)

    try:
            print(f"{team_1} vs {team_2} \n  {team_lst[win]} wins 🏆🏆")
            print(f"    {team_lst[win]} into the Quater-Finals ⏩⏩ \n")
            qualified_teams_1.append(team_lst[win])
    except IndexError:
            print(f"{team_1} vs {team_2} \n  Match Draw ⚽⚽") 
            winning_team=random.choice(team_lst)
            print(f"   {winning_team} wins at Penaly Shoot-Out 🥅🎯")
            print(f"    {winning_team} into the Quater-Finals ⏩⏩ \n")
            qualified_teams_1.append(winning_team)
    i=i+2
    
##Round of 16 Section_2
qualified_teams_2=[]
standings=list(Group_standings.keys())
i=0
while i < (len(standings)):
    A_team= sorted(Group_standings[standings[i]][3].items(), key=lambda x: x[1], reverse=True)
    team_1=A_team[1][0]
    B_team= sorted(Group_standings[standings[i+1]][3].items(), key=lambda x: x[1], reverse=True)
    team_2=B_team[0][0]
    
    #team_1_num=label_encoder.transform([team_1])[0]
    #team_2_num=label_encoder.transform([team_2])[0]
    team_lst=[team_1,team_2]
    
    Input_vector=np.array([[year,host_num,team_1_num,team_2_num]])
    res=model.predict(Input_vector)
    win,_=select_winning_team(res)

    try:
            print(f"{team_1} vs {team_2} \n  {team_lst[win]} wins 🏆🏆")
            print(f"    {team_lst[win]} into the Quater-Finals ⏩⏩ \n")
            qualified_teams_2.append(team_lst[win])
            
    except IndexError:
            print(f"{team_1} vs {team_2} \n  Match Draw ⚽⚽") 
            winning_team=random.choice(team_lst)
            print(f"   {winning_team} wins at Penaly Shoot-Out 🥅🎯")
            print(f"    {winning_team} into the Quater-Finals ⏩⏩ \n")
            qualified_teams_2.append(winning_team)
    i=i+2

Round of 16



NameError: ignored

accuracy 계산 함수